# Summary
This notebook is used to automatically extract data from ATB yearly workbook file and generate summary accordingly to the specification.

## Getting Ready
- Here, we first import the package needed, and identify the path to files;
    - 2 paths to files are required here:
        1. raw_data_path: the path to the raw data file
        2. ancillary_path: the path to the ancillary file which declares the scope of tech and index of interest.

- Make 2 lists out of the ancillary file

In [77]:
import pandas as pd

In [78]:
raw_data_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/2022 v2 Annual Technology Baseline Workbook Corrected 7-21-2022.xlsx'

ancillary_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/ancillary.xlsx'

raw = pd.ExcelFile(raw_data_path)

In [79]:
ancillary = pd.read_excel(ancillary_path)

# make tech list out of ancillary
tech_list = list(ancillary['Tech'].values)

# make index list out of ancillary
index_list = list(ancillary['Index'].dropna().values)

# flush-flush
del ancillary

## Main loop
- Iterate over tech sheets needed, and get rid of the extraneous rows.
- Make a dictionary of each sheet to store their corresponding dataframe

In [80]:
# get all the name of the sheets -> intersect with tech list
sheet_list = list(set(raw.sheet_names) & set(tech_list))
sub_tech_list = list(set(tech_list)-set(sheet_list))

In [84]:
sheets = {}
last_i = None

# iterate through all sheets level techs
for sheet in sheet_list:
    # special specification for storages
    if not 'Storage' in sheet:
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 9:].dropna(how='all')
    else:
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 3:].dropna(how='all')
    index = None
    # iterate over rows
    for i in range(len(sheet_df)):
        # get the value on the current index column
        working_index = sheet_df.iloc[i, 0]

        if not pd.isna(working_index):
            sheet_df.iloc[i-1, 0] = None    # get rid of the header (year) row
            if working_index in index_list:
                index = working_index
                last_i = i
            else:
                index = None

        sheet_df.iloc[i, 0] = index

    # get the header
    header = list(sheet_df.iloc[last_i-1, :].values)
    header[0:3] = ['Index', 'Display Name', 'Scenario']
    sheet_df.columns = header

    sheet_df = sheet_df.dropna(how='any', subset=['Index']).reset_index(drop=True)
    sheets[str(sheet)] = sheet_df

- for those tech that is relatively more detailed, we shall first generate a dictionary of its correspondence to the parent sheet it belongs to.
- For now, we are trying to match the sub-techs with the sheet which has a name that is most similar to it.
- i.e.,

In [83]:
'Storage' in 'Utility-Scale Battery Storage'

True